In [8]:
import tensorflow as tf
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import random
from PIL import Image

from models.resnetv2 import *

In [10]:
def cost(logits, labels):
    with tf.name_scope('loss'):
        # cross_entropy = tf.losses.sparse_softmax_cross_entropy(labels=y_, logits=y_conv)
        cross_entropy = tf.losses.softmax_cross_entropy(onehot_labels=labels, logits=logits)
    cross_entropy_cost = tf.reduce_mean(cross_entropy)
    return cross_entropy_cost

In [11]:
features = tf.placeholder(tf.float32, [None, 64, 64, 3], name='X')
labels = tf.placeholder(tf.int64, [None, 190], name='Y')

logits = resNet50_reference(features)

cross_entropy = cost(logits, labels)

with tf.name_scope('adam_optimizer'):
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

graph_location = tempfile.mkdtemp()
print('Saving graph to: %s' % graph_location)
train_writer = tf.summary.FileWriter(graph_location)
train_writer.add_graph(tf.get_default_graph())

mini_batches = random_mini_batches(X_train, Y_train, mini_batch_size=32, seed=None)

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
        X_mini_batch, Y_mini_batch = mini_batches[np.random.randint(0, len(mini_batches))]
        train_step.run(feed_dict={features: X_mini_batch, labels: Y_mini_batch, keep_prob: 0.5, train_mode: True})

        if i % 20 == 0:
            train_cost = sess.run(cross_entropy, feed_dict={features: X_mini_batch,
                                  labels: Y_mini_batch, keep_prob: 1.0, train_mode: False})
            print('step %d, training cost %g' % (i, train_cost))

    saver.save(sess, self.model_save_path)


ValueError: Shapes (?, 6) and (?, 190) are incompatible